
# Transfer learning for EEG

This example shows how to train a neural network with supervision on TUH [2]
EEG data and transfer the model to NMT [3] EEG dataset. We follow the approach of [1]


In [1]:
# Authors: MJ Bayazi <mj.darvishi92@gmail.com>
#
# License: BSD (3-clause)


random_state = 2024
n_jobs = 1

## Loading and preprocessing the dataset

### Load  and save the raw recordings

Here we assume you already load and preprocess the raw recordings for both TUAB and NMT datasetsand saved the file in `TUAB_path' and 'NMT_path' respectively. To read more see this notebook [here](https://braindecode.org/stable/auto_examples/applied_examples/plot_tuh_eeg_corpus.html) 

### Load the preprocessed data

Now, we load a few recordings from the TUAB dataset. Running
this example with more recordings should yield better representations and
downstream classification performance.

In [2]:
TUAB_path = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/tuab/tuab_pp3_wzscore_20min'
NMT_path = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/NMT/nmt_pp3_wzscore_20min'

from braindecode.datautil.serialization import  load_concat_dataset

TUAB_ds = load_concat_dataset(TUAB_path, preload=False,
                            target_name=['pathological','age','gender'] ,#)
                            ids_to_load=range(20)
                            )

<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_types is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.pick_channels_regexp is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
<frozen importlib._bootstrap>:241: FutureWarning: mne.io.pick.channel_type is deprecated will be removed in 1.6, use documented public API instead. If no appropriate public API exists, please open an issue on GitHub.
/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/conda_envs/braindecode_v3/lib/python3.10/site-packages/moabb/pipelines/__init__.py:26: ModuleNotFoundError: Tensorflow is not installed. You won't be able to use these MOABB pipelines if you attempt to do so.
  warn(


In [3]:
import pandas as pd
print("target is being set to pathological clf")
target = TUAB_ds.description['pathological'].astype(int)
for d, y in zip(TUAB_ds.datasets, target):
    d.description['pathological'] = y
    d.target_name = 'pathological'
    d.target = d.description[d.target_name]
TUAB_ds.set_description(pd.DataFrame([d.description for d in TUAB_ds.datasets]), overwrite=True)

target is being set to pathological clf


### Splitting dataset into train, valid and test sets

We split the recordings by subject into train, validation and
testing sets.


In [4]:
# split based on train split from dataset
train_set = TUAB_ds.split('train')['True']
test_set = TUAB_ds.split('train')['False']

## Creating the model

We can now create the deep learning model. In this tutorial, we use DeepNet introduced in [4].

In [5]:
from braindecode.models import Deep4Net

n_chans = 21
n_classes = 2
input_window_samples = 6000
drop_prob = 0.5
cuda = True # Set to False if you don't have a GPU
n_start_chans = 25
final_conv_length = 1
n_chan_factor = 2
stride_before_pool = True
# input_window_samples =6000
model = Deep4Net(
            n_chans, n_classes,
            n_filters_time=n_start_chans,
            n_filters_spat=n_start_chans,
            input_window_samples=input_window_samples,
            n_filters_2=int(n_start_chans * n_chan_factor),
            n_filters_3=int(n_start_chans * (n_chan_factor ** 2.0)),
            n_filters_4=int(n_start_chans * (n_chan_factor ** 3.0)),
            final_conv_length=final_conv_length,
            stride_before_pool=stride_before_pool,
            drop_prob=drop_prob)
            # Send model to GPU
if cuda:
    model.cuda()

from braindecode.models.util import to_dense_prediction_model, get_output_shape

to_dense_prediction_model(model)

n_preds_per_input = get_output_shape(model, n_chans, input_window_samples)[2]

/home/mila/m/mohammad-javad.darvishi-bayasi/projects/bd_4_APD/braindecode/braindecode/models/base.py:23: UserWarning: Deep4Net: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
/home/mila/m/mohammad-javad.darvishi-bayasi/projects/bd_4_APD/braindecode/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +
/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/conda_envs/braindecode_v3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:86: FutureWarning: Function to_dense_prediction_model is deprecated; will be removed in version 1.0. Use EEGModuleMixin.to_dense_prediction_model method directly on the model object.
  warnings.warn(msg, category=FutureWarning)
/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/conda_envs/braindecode_v3/lib/python3.10/site-packages/sklearn/utils/deprecation.py:86:

### Extracting windows

We extract 60-s windows to be used in both datasets. We use a window size of 6000 samples.


In [6]:
from IPython.utils import io
from braindecode.datautil.windowers import create_fixed_length_windows

with io.capture_output() as captured:
        window_train_set = create_fixed_length_windows(train_set, 
                                                    start_offset_samples=0,
                                                    stop_offset_samples=None,
                                                    preload=True,
                                                    window_size_samples=input_window_samples,
                                                    window_stride_samples=n_preds_per_input,
                                                    drop_last_window=True,)
        
with io.capture_output() as captured:
        window_test_set = create_fixed_length_windows(test_set,
                                                    start_offset_samples=0,
                                                    stop_offset_samples=None,preload=False,
                                                    window_size_samples=input_window_samples,
                                                    window_stride_samples=n_preds_per_input,
                                                    drop_last_window=False,)

/home/mila/m/mohammad-javad.darvishi-bayasi/projects/bd_4_APD/braindecode/braindecode/datautil/windowers.py:4: UserWarning: datautil.windowers module is deprecated and is now under preprocessing.windowers, please use from import braindecode.preprocessing.windowers
  warn('datautil.windowers module is deprecated and is now under '


## defining the classifier


In [7]:
from braindecode import EEGClassifier
from braindecode.training.losses import CroppedLoss
import torch
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

weight_decay = 0.5 * 0.001
lr = 0.0625 * 0.01
n_epochs = 10
batch_size = 64

clf = EEGClassifier(
                model,
                cropped=True,
                classes=[0, 1],
                criterion=CroppedLoss,
                criterion__loss_function=torch.nn.functional.nll_loss,
                optimizer=torch.optim.AdamW,
                train_split=predefined_split(window_test_set), 
                optimizer__lr=lr,
                optimizer__weight_decay=weight_decay,
                iterator_train__shuffle=True,
                batch_size=batch_size,
                callbacks=[
                    "accuracy", "balanced_accuracy","f1",("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                device='cuda' if cuda else 'cpu',
                )

clf.initialize() # This is important!
print('classifier initialized')
print("Number of parameters = ", sum(p.numel() for p in model.parameters() if p.requires_grad))


classifier initialized
Number of parameters =  277052


## Training

We can now train our network on the TUAB. We use similar
hyperparameters as in [1]_, but reduce the number of epochs and
increase the learning rate to account for the smaller setting of
this example.



In [8]:
clf.fit(window_train_set, y=None, epochs=n_epochs)

Re-initializing module.
Re-initializing criterion because the following parameters were re-set: loss_function.
Re-initializing optimizer.
  epoch    train_accuracy    train_balanced_accuracy    train_f1    train_loss    valid_accuracy    valid_balanced_accuracy    valid_f1    valid_loss      lr     dur
-------  ----------------  -------------------------  ----------  ------------  ----------------  -------------------------  ----------  ------------  ------  ------
      1            0.7778                     0.5000      0.8750        0.8197            0.5000                     0.5000      0.6667        1.2531  0.0006  2.2410
      2            0.8333                     0.6250      0.9032        0.5774            0.5000                     0.5000      0.6667        1.0273  0.0006  1.8734
      3            0.9444                     0.9643      0.9630        0.4605            0.5000                     0.5000      0.6667        0.7824  0.0006  1.8630
      4            0.8889       

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  Deep4Net (Deep4Net)                      [1, 21, 6000]             [1, 2, 5794]              --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 21, 6000]             [1, 21, 6000, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 21, 6000, 1]          [1, 1, 6000, 21]          --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 6000, 21]          [1, 25, 5991, 1]          13,400                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 25, 5991, 1]          [1, 25, 5991, 1]          50                        --
  ├─Expression (conv_nonlin): 1-5 

## saving the model

In [9]:
result_path = '/home/mila/m/mohammad-javad.darvishi-bayasi/scratch/medical/eeg/tuab/results/'
path = result_path + "model_{}.pt".format(random_state)
torch.save(clf.module, path)
path = result_path + "state_dict_{}.pt".format(random_state)
torch.save(clf.module.state_dict(), path)

clf.save_params(f_params=result_path +'model.pkl', f_optimizer= result_path +'opt.pkl', f_history=result_path +'history.json')

## Load NMT dataset

In [10]:
import mne
mne.set_log_level('ERROR')

NMT_ds = load_concat_dataset(NMT_path, preload=False,
                            target_name=['pathological','age','gender'] ,#)
                            ids_to_load=range(200)
                            )

In [11]:
# split based on train split from dataset
train_set = NMT_ds.split('train')['True']
test_set = NMT_ds.split('train')['False']

In [12]:
import pandas as pd
print("target is being set to pathological clf")
target = NMT_ds.description['pathological'].astype(int)
for d, y in zip(NMT_ds.datasets, target):
    d.description['pathological'] = y
    d.target_name = 'pathological'
    d.target = d.description[d.target_name]
NMT_ds.set_description(pd.DataFrame([d.description for d in NMT_ds.datasets]), overwrite=True)

target is being set to pathological clf


### extracting windows

In [13]:
from IPython.utils import io
from braindecode.datautil.windowers import create_fixed_length_windows

with io.capture_output() as captured:
        window_train_set = create_fixed_length_windows(train_set, 
                                                    start_offset_samples=0,
                                                    stop_offset_samples=None,
                                                    preload=True,
                                                    window_size_samples=input_window_samples,
                                                    window_stride_samples=n_preds_per_input,
                                                    drop_last_window=True,)
with io.capture_output() as captured:
        window_test_set = create_fixed_length_windows(test_set,
                                                    start_offset_samples=0,
                                                    stop_offset_samples=None,preload=False,
                                                    window_size_samples=input_window_samples,
                                                    window_stride_samples=n_preds_per_input,
                                                    drop_last_window=False,)

## loading the pre trained model

In [14]:
load_path = result_path + 'state_dict_2024.pt'
state_dicts = torch.load(load_path) 
model.load_state_dict(state_dicts, strict= False)
print('pre-trained model loaded using pytorch')

## freeze layers ##
freez = False
if freez:
    for ii, (name, param) in enumerate(model.named_parameters()):
        # if 'temporal_block_0' in name or 'temporal_block_1' in name or 'temporal_block_2' in name or 'temporal_block_3' in name: # or 'temporal_block_5' in name or 'conv_classifier' in name:
        if not 'conv_classifier' in name:
            param.requires_grad = False
            print('param:', name, param.requires_grad)

pre-trained model loaded using pytorch


## fine tuning the model on NMT dataset

In [15]:
from braindecode import EEGClassifier
from braindecode.training.losses import CroppedLoss
import torch
from skorch.callbacks import LRScheduler
from skorch.helper import predefined_split

weight_decay = 0.5 * 0.001
lr = 0.0625 * 0.01
n_epochs = 10
batch_size = 64

clf = EEGClassifier(
                model,
                cropped=True,
                classes=[0, 1],
                criterion=CroppedLoss,
                criterion__loss_function=torch.nn.functional.nll_loss,
                optimizer=torch.optim.AdamW,
                train_split=predefined_split(window_test_set), 
                optimizer__lr=lr,
                optimizer__weight_decay=weight_decay,
                iterator_train__shuffle=True,
                batch_size=batch_size,
                callbacks=[
                    "accuracy", "balanced_accuracy","f1",("lr_scheduler", LRScheduler('CosineAnnealingLR', T_max=n_epochs - 1)),
                    ],
                device='cuda' if cuda else 'cpu',
                )

clf.initialize() # This is important!
print('classifier initialized')
print("Number of parameters = ", sum(p.numel() for p in model.parameters() if p.requires_grad))


classifier initialized
Number of parameters =  277052


In [16]:
clf.fit(window_train_set, y=None, epochs=n_epochs)

Re-initializing module.
Re-initializing criterion because the following parameters were re-set: loss_function.
Re-initializing optimizer.
  epoch    train_accuracy    train_balanced_accuracy    train_f1    train_loss    valid_accuracy    valid_balanced_accuracy    valid_f1    valid_loss      lr      dur
-------  ----------------  -------------------------  ----------  ------------  ----------------  -------------------------  ----------  ------------  ------  -------
      1            0.8287                     0.5000      0.0000        0.7474            0.5263                     0.5000      0.0000        1.1033  0.0006  11.4481
      2            0.7348                     0.7376      0.4894        0.4602            0.7895                     0.8000      0.8182        0.5255  0.0006  11.3550
      3            0.7956                     0.7615      0.5432        0.4352            0.7895                     0.7889      0.7778        0.5083  0.0006  10.9772
      4            0.8508  

<class 'braindecode.classifier.EEGClassifier'>[initialized](
  module_=============================================================================================================================================
  Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
  Deep4Net (Deep4Net)                      [1, 21, 6000]             [1, 2, 5794]              --                        --
  ├─Ensure4d (ensuredims): 1-1             [1, 21, 6000]             [1, 21, 6000, 1]          --                        --
  ├─Rearrange (dimshuffle): 1-2            [1, 21, 6000, 1]          [1, 1, 6000, 21]          --                        --
  ├─CombinedConv (conv_time_spat): 1-3     [1, 1, 6000, 21]          [1, 25, 5991, 1]          13,400                    --
  ├─BatchNorm2d (bnorm): 1-4               [1, 25, 5991, 1]          [1, 25, 5991, 1]          50                        --
  ├─Expression (conv_nonlin): 1-5 

## Conclusion

In this example, we used transfer learning (TL) as a way to learn
representations from a large EEG data and transfer to a smaller dataset. 


## References

.. [1] Darvishi-Bayazi, M. J., Ghaemi, M. S., Lesort, T., Arefin, M. R., Faubert, J., & Rish, I. (2024). Amplifying pathological detection in EEG signaling pathways through cross-dataset transfer learning. Computers in Biology and Medicine, 169, 107893.

.. [2] Shawki, N., Shadin, M. G., Elseify, T., Jakielaszek, L., Farkas, T., Persidsky, Y., ... & Picone, J. (2022). Correction to: The temple university hospital digital pathology corpus. In Signal Processing in Medicine and Biology: Emerging Trends in Research and Applications (pp. C1-C1). Cham: Springer International Publishing..

.. [3] Khan, H. A., Ul Ain, R., Kamboh, A. M., & Butt, H. T. (2022). The NMT scalp EEG dataset: an open-source annotated dataset of healthy and pathological EEG recordings for predictive modeling. Frontiers in neuroscience, 15, 755817.


